In [1]:
import sys
sys.path.append("..")


In [2]:
import shap
import numpy as np
import matplotlib.pyplot as plt
from src.explainability import (
    shap_explain_logistic,
    shap_explain_tree
)


In [3]:
import joblib

preprocessor = joblib.load("../models/preprocessor.pkl")
rf_model = joblib.load("../models/rf_best_model.pkl")
rf_model


d:\fraud-detection\.venv\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.8.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\fraud-detection\.venv\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.8.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


RandomForestClassifier(max_depth=10, random_state=42)

In [4]:
import pandas as pd

X_train = pd.read_csv("../data/processed/X_train_processed.csv")
y_train = pd.read_csv("../data/processed/y_train.csv").values.ravel()


In [5]:
feature_names = preprocessor.get_feature_names_out()
len(feature_names)


153

In [6]:
from src.explainability import shap_explain_tree

result = shap_explain_tree(rf_model, X_train.values, feature_names, sample_size=2000)
print(type(result), len(result))


<class 'tuple'> 4


In [7]:
explainer, shap_values, X_shap, feature_names = shap_explain_tree(
    rf_model,
    X_train.values,
    feature_names,
    sample_size=2000
)
print("SHAP explainer and values ready!")


SHAP explainer and values ready!


In [8]:
shap.summary_plot(
    shap_values[1],  # class = fraud
    X_shap,
    feature_names=feature_names,
    show=True
)


AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

In [ ]:
importances = rf_model.feature_importances_

fi = (
    pd.DataFrame({
        "feature": feature_names,
        "importance": importances
    })
    .sort_values(by="importance", ascending=False)
    .head(10)
)

fi
fi.plot(
    x="feature",
    y="importance",
    kind="barh",
    title="Top 10 Feature Importances (Random Forest)"
)
plt.gca().invert_yaxis()
plt.show()


ValueError: All arrays must be of the same length

In [ ]:
def plot_force(i):
    shap.force_plot(
        explainer.expected_value[1],
        shap_values[1][i],
        X_shap[i],
        feature_names=feature_names,
        matplotlib=True
    )


In [ ]:
plot_force(0)


In [ ]:
plot_force(10)


In [ ]:
plot_force(20)


BUSINESS INSIGHTS 

Key Fraud Drivers Identified

1️⃣ Transaction Velocity

High number of transactions in a short time window

Strong indicator of automated or bot-driven fraud

2️⃣ Time-Based Features

Transactions during unusual hours (late night / early morning)

Fraud activity clusters outside normal business hours

3️⃣ Country / IP Risk

Certain countries show disproportionately higher fraud rates

IP-to-country mismatch increases fraud probability

4️⃣ Purchase Value Patterns

Very small or very large transaction amounts

Fraudsters test cards with small values before large fraud

💼 Business Recommendations

✔ Introduce real-time velocity checks
✔ Flag transactions from high-risk countries
✔ Apply stricter verification during odd hours
✔ Use SHAP explanations to support manual fraud review

🎯 Why SHAP is Valuable for the Business

Transparent decision-making

Regulatory compliance

Trustworthy AI models

Faster fraud investigations